In [13]:
!pip install neurokit2

In [14]:
!pip install hrv-analysis
!pip install pyhrv

  Obtaining dependency information for pexpect<5.0.0,>=4.9.0 from https://files.pythonhosted.org/packages/9e/c3/059298687310d527a58bb01f3b1965787ee3b40dce76752eda8b44e9a2c5/pexpect-4.9.0-py2.py3-none-any.whl.metadata
  Using cached pexpect-4.9.0-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached pexpect-4.9.0-py2.py3-none-any.whl (63 kB)
  Attempting uninstall: pexpect
    Found existing installation: pexpect 4.8.0
ERROR: Cannot uninstall pexpect 4.8.0, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps pexpect==4.8.0'.


In [15]:
import tensorflow as tf
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import wfdb
import os
import ast
import neurokit2 as nk
import matplotlib
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

In [16]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [17]:
import warnings
#from scipy.interpolate import interp1d
#from scipy import signal
#from scipy.integrate import trapz
#import pyhrv.tools as tools
#import pyhrv.frequency_domain as fd

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [18]:
path = '/kaggle/input/ptb-xl-a-large-publicly-available2/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3/'

ecg_data = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')
scp_data = pd.read_csv(path + 'scp_statements.csv')

In [19]:
ecg_data[:5]

,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
ecg_id,,,,,,,,,,,,,,,,,,,,,
1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr
4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,...,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr
5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,...,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr


In [20]:
scp_data.columns

Index(['Unnamed: 0', 'description', 'diagnostic', 'form', 'rhythm',
       'diagnostic_class', 'diagnostic_subclass', 'Statement Category',
       'SCP-ECG Statement Description', 'AHA code', 'aECG REFID', 'CDISC Code',
       'DICOM Code'],
      dtype='object')

In [21]:
ecg_data_new = ecg_data.copy()
for i in ['nurse','site','device','recording_date','heart_axis','infarction_stadium1','infarction_stadium2',
         'validated_by_human','baseline_drift','validated_by','second_opinion','initial_autogenerated_report',
         'static_noise','burst_noise','extra_beats','strat_fold','pacemaker','height']:
    del ecg_data_new[i]


In [22]:
ecg_data_new[:5]

,patient_id,age,sex,weight,report,scp_codes,electrodes_problems,filename_lr,filename_hr
ecg_id,,,,,,,,,
1,15709.0,56.0,1,63.0,sinusrhythmus periphere niederspannung,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",NaN,records100/00000/00001_lr,records500/00000/00001_hr
2,13243.0,19.0,0,70.0,sinusbradykardie sonst normales ekg,"{'NORM': 80.0, 'SBRAD': 0.0}",NaN,records100/00000/00002_lr,records500/00000/00002_hr
3,20372.0,37.0,1,69.0,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",NaN,records100/00000/00003_lr,records500/00000/00003_hr
4,17014.0,24.0,0,82.0,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",NaN,records100/00000/00004_lr,records500/00000/00004_hr
5,17448.0,19.0,1,70.0,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",NaN,records100/00000/00005_lr,records500/00000/00005_hr


In [23]:
scp_data[:5]

,Unnamed: 0,description,diagnostic,form,rhythm,diagnostic_class,diagnostic_subclass,Statement Category,SCP-ECG Statement Description,AHA code,aECG REFID,CDISC Code,DICOM Code
0,NDT,non-diagnostic T abnormalities,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,non-diagnostic T abnormalities,NaN,NaN,NaN,NaN
1,NST_,non-specific ST changes,1.0,1.0,NaN,STTC,NST_,Basic roots for coding ST-T changes and abnorm...,non-specific ST changes,145.0,MDC_ECG_RHY_STHILOST,NaN,NaN
2,DIG,digitalis-effect,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,suggests digitalis-effect,205.0,NaN,NaN,NaN
3,LNGQT,long QT-interval,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,long QT-interval,148.0,NaN,NaN,NaN
4,NORM,normal ECG,1.0,NaN,NaN,NORM,NORM,Normal/abnormal,normal ECG,1.0,NaN,NaN,F-000B7


In [24]:
scp_data[(scp_data["rhythm"] ==1) & (scp_data["AHA code"] > 0)]

,Unnamed: 0,description,diagnostic,form,rhythm,diagnostic_class,diagnostic_subclass,Statement Category,SCP-ECG Statement Description,AHA code,aECG REFID,CDISC Code,DICOM Code
59,SR,sinus rhythm,NaN,NaN,1.0,NaN,NaN,Statements related to impulse formation (abnor...,sinus rhythm,20.0,MDC_ECG_RHY_SINUS_RHY,NaN,NaN
60,AFIB,atrial fibrillation,NaN,NaN,1.0,NaN,NaN,Statements related to impulse formation (abnor...,atrial fibrillation,50.0,MDC_ECG_RHY_ATR_FIB,NaN,D3-31520
61,STACH,sinus tachycardia,NaN,NaN,1.0,NaN,NaN,Statements related to impulse formation (abnor...,sinus tachycardia,21.0,MDC_ECG_RHY_SINUS_TACHY,NaN,NaN
62,SARRH,sinus arrhythmia,NaN,NaN,1.0,NaN,NaN,Statements related to impulse formation (abnor...,sinus arrhythmia,23.0,MDC_ECG_RHY_SINUS_ARRHY,NaN,NaN
63,SBRAD,sinus bradycardia,NaN,NaN,1.0,NaN,NaN,Statements related to impulse formation (abnor...,sinus bradycardia,22.0,MDC_ECG_RHY_SINUS_BRADY,NaN,NaN
67,AFLT,atrial flutter,NaN,NaN,1.0,NaN,NaN,Statements related to impulse formation (abnor...,atrial flutter,51.0,MDC_ECG_RHY_ATR_FLUT,NaN,NaN
68,SVTAC,supraventricular tachycardia,NaN,NaN,1.0,NaN,NaN,Statements related to impulse formation (abnor...,supraventricular tachycardia,55.0,MDC_ECG_RHY_SV_TACHY,NaN,D3-31290


In [25]:
def del_symbol_l_1(ecg_data_new_del):
    for i in ecg_data_new_del.index:
        str_0 = ecg_data_new_del.scp_codes[i][1:-1] + ','
        str_0 = str_0.replace("'", "")
        ecg_data_new_del.scp_codes[i] = str_0
    return ecg_data_new_del
ecg_data_new  = del_symbol_l_1(ecg_data_new)
ecg_data_new[:2]

,patient_id,age,sex,weight,report,scp_codes,electrodes_problems,filename_lr,filename_hr
ecg_id,,,,,,,,,
1,15709.0,56.0,1,63.0,sinusrhythmus periphere niederspannung,"NORM: 100.0, LVOLT: 0.0, SR: 0.0,",NaN,records100/00000/00001_lr,records500/00000/00001_hr
2,13243.0,19.0,0,70.0,sinusbradykardie sonst normales ekg,"NORM: 80.0, SBRAD: 0.0,",NaN,records100/00000/00002_lr,records500/00000/00002_hr


## Выделяем нарушения ритма

In [26]:
def del_symbol_l_2_2(ecg_data_new_del,scp_data_n,name1,name2):
    list1=[]
    list2=[]
    for i in ecg_data_new_del.index:
        str_0 = ""
        rhythm_number=""
        for j in ecg_data_new_del.scp_codes[i].split(","):
            if  j.split(":")[0] in list(scp_data_n["Unnamed: 0"]) :
                str_0= str_0+ j.split(":")[0] +","
                rhythm_number= rhythm_number+ j.split(":")[1] +","
            if  j.split(":")[0][1:] in list(scp_data_n["Unnamed: 0"]):
                j = j[1:]
                str_0= str_0+ j.split(":")[0] +","
                rhythm_number= rhythm_number+ j.split(":")[1] +","
        
        list1.append(str_0[:-1])
        list2.append(rhythm_number[:-1])
    ecg_data_new_del[name1] = list1
    ecg_data_new_del[name2] = list2
    return ecg_data_new_del

In [27]:
scp_data_n = scp_data[(scp_data["rhythm"] ==1) & (scp_data["AHA code"] > 0)]
ecg_data_new = del_symbol_l_2_2(ecg_data_new,scp_data_n,"rhythm","rhythm_number")

In [28]:
ecg_data_new.rhythm.value_counts()#.plot(kind='bar');

rhythm
SR             16739
AFIB            1496
                1243
STACH            821
SARRH            767
SBRAD            634
AFLT              40
SVTAC             18
AFLT,AFIB         15
AFLT,SVTAC         6
AFLT,STACH         5
SBRAD,SARRH        3
AFLT,SR            3
SR,AFLT            2
AFIB,AFLT          2
SR,SARRH           2
SVTAC,AFIB         1
SR,SVTAC           1
SVTAC,SR           1
Name: count, dtype: int64

In [29]:
ecg_data_new.rhythm.value_counts().index[:8]

Index(['SR', 'AFIB', '', 'STACH', 'SARRH', 'SBRAD', 'AFLT', 'SVTAC'], dtype='object', name='rhythm')

In [30]:
for i in ecg_data_new.index:
    if (ecg_data_new.rhythm[i] in ecg_data_new.rhythm.value_counts().index[:8]) != True:
        ecg_data_new = ecg_data_new.drop(index=[i])

In [31]:
ecg_data_new.rhythm.value_counts()

rhythm
SR       16739
AFIB      1496
          1243
STACH      821
SARRH      767
SBRAD      634
AFLT        40
SVTAC       18
Name: count, dtype: int64

В основном выделен:
1. SR -  синусовый ритм 
2. AFIB - atrial fibrillation,
3. [] - не заполненно? 
4. STACH - sinus tachycardia
5. SARRH - sinus arrhythmia
6. SBRAD - sinus bradycardia
7. AFLT(40) - atrial flutter
8. SVTAC(18) - supraventricular tachycardia

Не имеет смысла рассматривать двойные дагнозы ритма

## Выделяем нарушения диагноза

In [32]:
ecg_data_new_12  = del_symbol_l_2_2(ecg_data_new,scp_data[(scp_data["diagnostic"] == 1)& (scp_data["form"] != 1)],"diagnostic","diagnostic_namber")
len(ecg_data_new_12[ecg_data_new_12.diagnostic != ""])

19573

In [33]:
ecg_data_new_12[ecg_data_new_12.diagnostic == "IMI"].rhythm.value_counts()

rhythm
SR       898
AFIB      80
SARRH     51
STACH     46
          45
SBRAD     24
AFLT       2
SVTAC      2
Name: count, dtype: int64

In [34]:
ecg_data_new_12

,patient_id,age,sex,weight,report,scp_codes,electrodes_problems,filename_lr,filename_hr,rhythm,rhythm_number,diagnostic,diagnostic_namber
ecg_id,,,,,,,,,,,,,
1,15709.0,56.0,1,63.0,sinusrhythmus periphere niederspannung,"NORM: 100.0, LVOLT: 0.0, SR: 0.0,",NaN,records100/00000/00001_lr,records500/00000/00001_hr,SR,0.0,NORM,100.0
2,13243.0,19.0,0,70.0,sinusbradykardie sonst normales ekg,"NORM: 80.0, SBRAD: 0.0,",NaN,records100/00000/00002_lr,records500/00000/00002_hr,SBRAD,0.0,NORM,80.0
3,20372.0,37.0,1,69.0,sinusrhythmus normales ekg,"NORM: 100.0, SR: 0.0,",NaN,records100/00000/00003_lr,records500/00000/00003_hr,SR,0.0,NORM,100.0
4,17014.0,24.0,0,82.0,sinusrhythmus normales ekg,"NORM: 100.0, SR: 0.0,",NaN,records100/00000/00004_lr,records500/00000/00004_hr,SR,0.0,NORM,100.0
5,17448.0,19.0,1,70.0,sinusrhythmus normales ekg,"NORM: 100.0, SR: 0.0,",NaN,records100/00000/00005_lr,records500/00000/00005_hr,SR,0.0,NORM,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21833,17180.0,67.0,1,NaN,ventrikulÄre extrasystole(n) sinustachykardie ...,"NDT: 100.0, PVC: 100.0, VCLVH: 0.0, STACH: 0.0,",NaN,records100/21000/21833_lr,records500/21000/21833_hr,STACH,0.0,,
21834,20703.0,300.0,0,NaN,sinusrhythmus lagetyp normal qrs(t) abnorm ...,"NORM: 100.0, ABQRS: 0.0, SR: 0.0,",NaN,records100/21000/21834_lr,records500/21000/21834_hr,SR,0.0,NORM,100.0
21835,19311.0,59.0,1,NaN,sinusrhythmus lagetyp normal t abnorm in anter...,"ISCAS: 50.0, SR: 0.0,",NaN,records100/21000/21835_lr,records500/21000/21835_hr,SR,0.0,ISCAS,50.0


# **Второй датасет a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study**

In [37]:
import os
# Указываем путь к директории
directory = "/kaggle/input/chapman-ecg-database/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0/WFDBRecords/"
list_path_file = os.listdir(directory)
path_file_new=[]
for i in list_path_file:
    directory_2 = directory + i + "/"
    for j in os.listdir(directory_2):
        path_file_new.append( i + "/" + j )
path_file_new_2 = []    
ecg_id=[]
filename=[]
for i in path_file_new:
    directory_2 = directory + i + "/"
    for j in os.listdir(directory_2):
        j = j.replace(".hea", "")
        j = j.replace(".mat", "")
        if j!="RECORDS":
            if j in filename:
                f=0
            else:
                filename.append(str(j))
                path_file_new_2.append(i+ "/")
            #+ j
                ecg_id.append(int(j[2:]))
            
path_file_new_2.sort()   
ecg_id.sort()
filename.sort()
path_file_new_2 = pd.DataFrame({"ecg_id":ecg_id,"filename_path":path_file_new_2,"filename":filename,})

In [38]:
path_file_new_2[:2]

,ecg_id,filename_path,filename
0,1,01/010/,JS00001
1,2,01/010/,JS00002


In [68]:
def load_diagnos_Dx(df, path,scp_data):
    Age = []
    Sex = []
    Dx = []
    for i in df.index:
        data = [wfdb.rdsamp(os.path.join(path+"WFDBRecords/" + df.filename_path[int(i)], df.filename[int(i)]))]
        #data = np.array([signal for signal, meta in data])
        data_hea_comments = list(pd.DataFrame(data)[1])[0]['comments']
        Age.append(data_hea_comments[0][len("Age: "):len(data_hea_comments[0])])
        Sex.append(data_hea_comments[1][len("Sex: "):len(data_hea_comments[1])])
        Dx.append(data_hea_comments[2][len("Dx: "):len(data_hea_comments[2])])
    df["Age"] = Age
    df["Sex"] = Sex
    df["Dx"] = Dx
    return df

In [40]:
ecg_data_2 = path_file_new_2.copy()

In [41]:
path_2 = '/kaggle/input/chapman-ecg-database/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0/'
scp_data_2 = pd.read_csv(path_2 + 'ConditionNames_SNOMED-CT.csv')

In [42]:
scp_data_2_AcronymName = scp_data_2["Acronym Name"].copy()
#for i in scp_data_2.index:
#    if len(scp_data[scp_data["Unnamed: 0"]== scp_data_2["Acronym Name"][i]]) == 1:
#        scp_data_2_AcronymName[i] ="Да"
scp_data_2["scp_data"] = scp_data_2_AcronymName
#lengh = scp_data_2[scp_data_2["scp_data"] != "Да"].index
lengh=[ 2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 55, 56, 57, 58, 59, 60, 61,
       62]
scp_data_2.scp_data.loc[lengh[0]] = scp_data_2["Acronym Name"].loc[lengh[0]][:-1]
scp_data_2.scp_data.loc[lengh[1]] = scp_data_2["Acronym Name"].loc[lengh[1]][:-1]
scp_data_2.scp_data.loc[lengh[4]] = "PAC"
scp_data_2.scp_data.loc[lengh[16]] = "CLBBB"
scp_data_2.scp_data.loc[lengh[17]] = "CLBBB"
scp_data_2.scp_data.loc[lengh[18]] = "CLBBB"
scp_data_2.scp_data.loc[lengh[23]] = "PMI"
scp_data_2.scp_data.loc[lengh[24]] = "AMI"
scp_data_2.scp_data.loc[lengh[25]] = "IMI"
scp_data_2.scp_data.loc[lengh[26]] = "LMI"
scp_data_2.scp_data.loc[lengh[42]] = "PVC"
scp_data_2.scp_data.loc[lengh[46]] = "SBRAD"
scp_data_2.scp_data.loc[lengh[47]] = "STACH"
scp_data_2.scp_data.loc[lengh[48]] = "AFLT"
scp_data_2.scp_data.loc[lengh[50]] = "SVTAC"
scp_data_2["Acronym Name"] = scp_data_2["scp_data"]

In [58]:
scp_data_2[:2000]

,Acronym Name,Full Name,Snomed_CT,scp_data
0,1AVB,1 degree atrioventricular block,270492004,1AVB
1,2AVB,2 degree atrioventricular block,195042002,2AVB
2,2AVB,2 degree atrioventricular block(Type one),54016002,2AVB
3,2AVB,2 degree atrioventricular block(Type two),28189009,2AVB
4,3AVB,3 degree atrioventricular block,27885002,3AVB
...,...,...,...,...
58,SVTAC,Supraventricular Tachycardia,426761007,SVTAC
59,AT,Atrial Tachycardia,713422000,AT
60,AVNRT,Atrioventricular Node Reentrant Tachycardia,233896004,AVNRT
61,AVRT,Atrioventricular Reentrant Tachycardia,233897008,AVRT


In [ ]:
#ecg_data_2 = ecg_data_2.drop(labels = [999],axis = 0)
#ecg_data_2 = ecg_data_2.drop(labels = [22674],axis = 0)
#ecg_data_2 = ecg_data_2.drop(labels = [999],axis = 0)

Заргузка кодов из .hea 

In [56]:
ecg_data_2[:2]

,ecg_id,filename_path,filename
0,1,01/010/,JS00001
1,2,01/010/,JS00002


In [59]:
ecg_data_2[998:1002]

,ecg_id,filename_path,filename
998,1051,01/019/,JS01051
999,1052,01/019/,JS01052
1000,1053,02/020/,JS01053
1001,1054,02/020/,JS01054


In [85]:
ecg_data_2_df = load_diagnos_Dx(ecg_data_2, path_2,scp_data_2)

In [87]:
ecg_data_2_df[:2]

,ecg_id,filename_path,filename,Age,Sex,Dx
0,1,01/010/,JS00001,85.0,Male,"164889003,59118001,164934002"
1,2,01/010/,JS00002,59.0,Female,"426177001,164934002"


In [88]:
# Сохранили и загрузили файл для последующей работы из буфера
ecg_data_2_df = pd.read_csv("/kaggle/input/electrocardiogram-database-for-arrhythmia-study/ecg_data_2.csv",index_col="Unnamed: 0")

Расшифровка

In [89]:
Dx_str= ecg_data_2_df.Dx
Dx_str_new=[]
for i in Dx_str:
    Dx_str_CT= ""
    i=i.split(",")
    for j in i:
        if len(str(list(scp_data_2[scp_data_2.Snomed_CT== int(j)]["scp_data"]))[1:-1])>1:
            Dx_str_CT += str(list(scp_data_2[scp_data_2.Snomed_CT== int(j)]["scp_data"]))[2:-2] +","
    Dx_str_new.append(Dx_str_CT[:-1])

In [90]:
ecg_data_2_df["diagnos"] = Dx_str_new

In [91]:
ecg_data_2_df[:2]

,ecg_id,filename_path,filename,Age,Sex,Dx,diagnos
0,1,01/010/,JS00001,85.0,Male,"164889003,59118001,164934002","AFIB,RBBB,TWC"
1,2,01/010/,JS00002,59.0,Female,"426177001,164934002","SBRAD,TWC"


In [92]:
ecg_data_new_12[:2]

,patient_id,age,sex,weight,report,scp_codes,electrodes_problems,filename_lr,filename_hr,rhythm,rhythm_number,diagnostic,diagnostic_namber
ecg_id,,,,,,,,,,,,,
1,15709.0,56.0,1,63.0,sinusrhythmus periphere niederspannung,"NORM: 100.0, LVOLT: 0.0, SR: 0.0,",NaN,records100/00000/00001_lr,records500/00000/00001_hr,SR,0.0,NORM,100.0
2,13243.0,19.0,0,70.0,sinusbradykardie sonst normales ekg,"NORM: 80.0, SBRAD: 0.0,",NaN,records100/00000/00002_lr,records500/00000/00002_hr,SBRAD,0.0,NORM,80.0


Разделение на диагнозы и нарушения ритма

In [93]:
def f_rhythm_diagnostic(ecg_data_2_df , name):
    df_l=[]
    for i in ecg_data_2_df.index:
        l_ = ""
        for j in ecg_data_2_df.diagnos[i].split(","):
            if str(j) in list(ecg_data_new_12[name].value_counts().index) or str(j) in list(ecg_data_new_12[name].value_counts().index) :
                l_ = l_ + j +","
        df_l.append(l_[:-1])
    return df_l
ecg_data_2_df["rhythm"] = f_rhythm_diagnostic(ecg_data_2_df , "rhythm")
ecg_data_2_df["diagnostic"] = f_rhythm_diagnostic(ecg_data_2_df , "diagnostic")

In [94]:
ecg_data_2_df[:2]

,ecg_id,filename_path,filename,Age,Sex,Dx,diagnos,rhythm,diagnostic
0,1,01/010/,JS00001,85.0,Male,"164889003,59118001,164934002","AFIB,RBBB,TWC",AFIB,
1,2,01/010/,JS00002,59.0,Female,"426177001,164934002","SBRAD,TWC",SBRAD,


In [95]:
ecg_data_2_df.rhythm.value_counts()

rhythm
SBRAD          16552
SR              8099
AFLT            7984
STACH           7239
                2736
AFIB            1780
SVTAC            672
SVTAC,AFLT        43
SR,AFLT           15
SBRAD,SR           6
AFLT,SR            5
STACH,AFLT         5
AFLT,STACH         4
STACH,SVTAC        3
SVTAC,STACH        3
AFLT,SVTAC         3
SBRAD,AFLT         1
Name: count, dtype: int64

In [96]:
ecg_data_2_df.diagnostic.value_counts()

diagnostic
                43050
1AVB             1116
LVH               617
RVH                99
2AVB               86
WPW                72
3AVB               69
1AVB,LVH           15
1AVB,2AVB           6
LVH,2AVB            5
RVH,LVH             5
3AVB,LVH            4
RVH,3AVB            3
RVH,1AVB            2
RVH,1AVB,LVH        1
Name: count, dtype: int64

In [ ]:
#ecg_data_2_df.to_csv("ecg_data_2_df.csv")

Обьеденение 2-х датасетов для выявления общего количиства лиагнозов и ритма

In [104]:
pd.concat([ecg_data_new_12.rhythm,
ecg_data_2_df.rhythm]).value_counts()

rhythm
SR             24838
SBRAD          17186
STACH           8060
AFLT            8024
                3979
AFIB            3276
SARRH            767
SVTAC            690
SVTAC,AFLT        43
SR,AFLT           15
SBRAD,SR           6
AFLT,SR            5
STACH,AFLT         5
AFLT,STACH         4
STACH,SVTAC        3
SVTAC,STACH        3
AFLT,SVTAC         3
SBRAD,AFLT         1
Name: count, dtype: int64

In [106]:
pd.concat([ecg_data_new_12.diagnostic,
ecg_data_2_df.diagnostic]).value_counts()[:20]

diagnostic
                45235
NORM             9094
1AVB             1223
LVH              1192
IMI              1148
LAFB              525
LVH,ISC_          459
ASMI              441
IRBBB             361
CLBBB             359
NORM,IRBBB        243
ILMI,LMI          182
ASMI,LAFB         170
IVCD              153
IMI,ASMI          150
WPW               142
ASMI,IMI          139
ISCAL             136
IMI,LVH,ISC_      111
CRBBB             109
Name: count, dtype: int64

In [102]:
ecg_data_2_df.to_csv("df_arrhytmia.csv")
ecg_data_new_12.to_csv("df_ptb_xl.csv")

Удаление шумовых данных

In [107]:
ecg_data_new_12[5:10] # ecg_data

,patient_id,age,sex,weight,report,scp_codes,electrodes_problems,filename_lr,filename_hr,rhythm,rhythm_number,diagnostic,diagnostic_namber
ecg_id,,,,,,,,,,,,,
6,19005.0,18.0,1,58.0,sinusrhythmus normales ekg,"NORM: 100.0, SR: 0.0,",NaN,records100/00000/00006_lr,records500/00000/00006_hr,SR,0.0,NORM,100.0
7,16193.0,54.0,0,83.0,"sinusrhythmus linkstyp t abnormal, wahrscheinl...","NORM: 100.0, SR: 0.0,",NaN,records100/00000/00007_lr,records500/00000/00007_hr,SR,0.0,NORM,100.0
8,11275.0,48.0,0,95.0,sinusrhythmus linkstyp qrs(t) abnormal infe...,"IMI: 35.0, ABQRS: 0.0, SR: 0.0,",NaN,records100/00000/00008_lr,records500/00000/00008_hr,SR,0.0,IMI,35.0
9,18792.0,55.0,0,70.0,sinusrhythmus normales ekg,"NORM: 100.0, SR: 0.0,",NaN,records100/00000/00009_lr,records500/00000/00009_hr,SR,0.0,NORM,100.0
10,9456.0,22.0,1,56.0,sinusrhythmus normales ekg,"NORM: 100.0, SR: 0.0,",NaN,records100/00000/00010_lr,records500/00000/00010_hr,SR,0.0,NORM,100.0


In [108]:
path = '/kaggle/input/ptb-xl-a-large-publicly-available2/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3/'
ecg_data = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')
scp_data = pd.read_csv(path + 'scp_statements.csv', index_col=0)

,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
ecg_id,,,,,,,,,,,,,,,,,,,,,
1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr
4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,...,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr
5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,...,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21833,17180.0,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-05-31 09:14:35,ventrikulÄre extrasystole(n) sinustachykardie ...,...,True,NaN,", alles,",NaN,NaN,1ES,NaN,7,records100/21000/21833_lr,records500/21000/21833_hr
21834,20703.0,300.0,0,NaN,NaN,1.0,2.0,AT-60 3,2001-06-05 11:33:39,sinusrhythmus lagetyp normal qrs(t) abnorm ...,...,True,NaN,NaN,NaN,NaN,NaN,NaN,4,records100/21000/21834_lr,records500/21000/21834_hr
21835,19311.0,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-06-08 10:30:27,sinusrhythmus lagetyp normal t abnorm in anter...,...,True,NaN,", I-AVR,",NaN,NaN,NaN,NaN,2,records100/21000/21835_lr,records500/21000/21835_hr


In [111]:
ecg_data.electrodes_problems.value_counts()

electrodes_problems
V6                          8
V4                          5
V1                          4
V1???                       2
V5                          2
V3                          1
V5,V6                       1
Kontaktprobleme aVL ???     1
I???                        1
aVL???                      1
v6????                      1
Elektroden vertauscht???    1
v4, v5                      1
V3,V4-V6                    1
Name: count, dtype: int64

In [115]:
for i in ecg_data_new_12.index:
    if ecg_data_new_12.electrodes_problems[i] in ecg_data_new_12.electrodes_problems.value_counts().index:
        ecg_data_new_12 = ecg_data_new_12.drop(labels = [i],axis = 0)
ecg_data_ptb_xl =ecg_data_new_12[(ecg_data_new_12.diagnostic != "") & (ecg_data_new_12.rhythm != "")]

In [116]:
ecg_data_ptb_xl.electrodes_problems.value_counts()

Series([], Name: count, dtype: int64)

In [ ]:
ecg_data_ptb_xl.to_csv("df_ptb_xl.csv")

# mimic 4

In [117]:
path = '/kaggle/input/ptb-xl-a-large-publicly-available2/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3/'

ecg_data = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')
scp_data = pd.read_csv(path + 'scp_statements.csv')
scp_data[scp_data.diagnostic == 1].diagnostic_class.value_counts()

diagnostic_class
MI      14
STTC    13
CD      11
HYP      5
NORM     1
Name: count, dtype: int64

In [118]:
machine_measurements = pd.read_csv("/kaggle/input/electrocardiogram-database-for-arrhythmia-study/mimic/machine_measurements.csv", index_col=0)
machine_measurements.report_0.value_counts()

/tmp/ipykernel_42/2419392448.py:1: DtypeWarning: Columns (16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  machine_measurements = pd.read_csv("/kaggle/input/electrocardiogram-database-for-arrhythmia-study/mimic/machine_measurements.csv", index_col=0)


report_0
Sinus rhythm                                                                                                                          317278
Sinus rhythm.                                                                                                                          59739
Sinus bradycardia                                                                                                                      58924
Sinus tachycardia                                                                                                                      40528
Atrial fibrillation                                                                                                                    31961
                                                                                                                                       ...  
Sinus tachycardia with sinus arrhythmia with fusion complexes                                                                              1
Poss

In [119]:
machine_measurements.report_1.value_counts()

report_1
Left axis deviation                                                              62179
Prolonged QT interval                                                            30202
Leftward axis                                                                    28239
Poor R wave progression - probable normal variant                                22837
Right bundle branch block                                                        18877
                                                                                 ...  
Extensive ST elevation, consider recent extensive infarction                         1
Lead(s) unsuitable for analysis: II III aVR aVL aVF V1 V2 V5 V6                      1
Lead(s) unsuitable for analysis: II III aVR aVL aVF V2 V5 V6                         1
Lead(s) unsuitable for analysis: II III aVR aVF V5                                   1
Sinus tachycardia with sinus arrhythmia with borderline 1st degree A-V block.        1
Name: count, Length: 2254, dtype: 

In [120]:
for i in ['report_0','report_1','report_2','report_3','report_4','report_5','report_6','report_7','report_8','report_9']:
    machine_measurements[i] = machine_measurements[i].replace(to_replace ='[.]', value = '', regex = True).str.lower()

In [121]:
machine_measurements = machine_measurements[machine_measurements.report_2 != "normal ecg"]
machine_measurements = machine_measurements[machine_measurements.report_2 != "normal ecg except for rate"]

In [122]:
machine_measurements[machine_measurements.report_2 == "normal ecg except for rate"].report_9.value_counts()[:20]

Series([], Name: count, dtype: int64)

In [123]:
scp_data[(scp_data.rhythm == 1) | (scp_data.diagnostic == 1)][:2]

,Unnamed: 0,description,diagnostic,form,rhythm,diagnostic_class,diagnostic_subclass,Statement Category,SCP-ECG Statement Description,AHA code,aECG REFID,CDISC Code,DICOM Code
0,NDT,non-diagnostic T abnormalities,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,non-diagnostic T abnormalities,NaN,NaN,NaN,NaN
1,NST_,non-specific ST changes,1.0,1.0,NaN,STTC,NST_,Basic roots for coding ST-T changes and abnorm...,non-specific ST changes,145.0,MDC_ECG_RHY_STHILOST,NaN,NaN


In [124]:
machine_measurements = machine_measurements.dropna(subset=["report_0"])
for i in machine_measurements.report_0.index:
    if isinstance(machine_measurements.report_0[i], float):
        print(machine_measurements.report_0[i])

In [125]:
descript = list(scp_data[(scp_data.rhythm == 1)].description)
#Unnamed = list(scp_data[(scp_data.rhythm == 1)]["Unnamed: 0"])
scp_data_new = scp_data[(scp_data.diagnostic == 1)][["Unnamed: 0","description"]].copy()
scp_data_new["machine_measurements"] = ["-"]*len(scp_data_new)

In [126]:
def f_1(l_new,name):
    pand  = pd.DataFrame({"report":[],"report_1":[],"study_id":[]})
    for i in l_new:
        df = machine_measurements[machine_measurements.report_1 == i].copy()
        study_id =df.study_id
        report=[name]* len(df.study_id)
        report_1=[i]* len(df.study_id)
        pand = pd.concat([
            pand,
        pd.DataFrame({"report":report,"report_1":report_1,"study_id":study_id})
        ])
    return pand
def func_poisk_name(osn,osn_1,iskl ):
    report = "report_1"
    report_1 = pd.DataFrame({"index" : machine_measurements[report].value_counts().to_frame().index ,"counts" : list(machine_measurements[report].value_counts())})
    l_new = report_1[report_1["index"].str.contains(osn)]
    for i in osn_1:
        l_new = l_new[l_new["index"].str.contains(i)]
    for i in iskl:
        l_new = l_new[(l_new["index"].str.contains(i))!=True]
    l_new = list(l_new[:10]["index"])
    return l_new

In [128]:
scp_data_new["machine_measurements"] = ["-"]*len(scp_data_new)
for i in scp_data_new.index:
    if len(func_poisk_name(scp_data_new.description[i],[],[] ))>0:
        scp_data_new["machine_measurements"][i] = func_poisk_name(scp_data_new.description[i],[],[] )

/tmp/ipykernel_42/1696149271.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  l_new = report_1[report_1["index"].str.contains(osn)]
/tmp/ipykernel_42/1696149271.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  l_new = report_1[report_1["index"].str.contains(osn)]


In [129]:
df_new_machine_measurements = pd.DataFrame({"report":[],"report_1":[],"study_id":[]})
for i in scp_data_new[scp_data_new["machine_measurements"] != "-"].index:
    jn = scp_data_new[scp_data_new["machine_measurements"] != "-"].machine_measurements[i]
    k = scp_data_new[scp_data_new["machine_measurements"] != "-"]["Unnamed: 0"][i]
    df_new_machine_measurements = pd.concat([df_new_machine_measurements,f_1(jn,k)])

In [130]:
df_new_machine_measurements.report.value_counts()

report
LAFB     19364
LVH      14868
RVH       1025
IRBBB      573
CRBBB      573
LPFB       410
CLBBB      406
ILBBB      406
AMI          1
Name: count, dtype: int64

Поиск по ключевым словам 

In [131]:
 df_new_machine_measurements = pd.concat([df_new_machine_measurements,
                                          f_1(func_poisk_name("a-v block",["1st"],[])+func_poisk_name("a-v block",["first"],[]),"1AVB")])

In [132]:
 df_new_machine_measurements = pd.concat([df_new_machine_measurements,
                                          f_1(func_poisk_name("a-v block",["3"],[])+func_poisk_name("a-v block",["2"],[]),"2AVB")])

In [133]:
df_new_machine_measurements = pd.concat([df_new_machine_measurements,
                                          f_1(func_poisk_name("myocard",["infer","lateral"],["ischemia"] ),"ILMI")])

In [134]:
df_new_machine_measurements = pd.concat([df_new_machine_measurements,
                                          f_1(func_poisk_name("myocard",["anter","lateral"],["ischemia"] ),"ALMI")])

In [135]:
df_new_machine_measurements = pd.concat([df_new_machine_measurements,
                                          f_1(func_poisk_name("wolf",[],[] ),"WPW")])

In [136]:
df_new_machine_measurements = pd.concat([df_new_machine_measurements,
                                          f_1(func_poisk_name("ischemi",["lateral"],["infero","antero","myocardial"] ),"ISCLA")])

In [137]:
df_new_machine_measurements = pd.concat([df_new_machine_measurements,
                                          f_1(func_poisk_name("ischemi",["infer","lateral"],["antero","myocardial"] ),"ISCIL")])

In [138]:
df_new_machine_measurements = pd.concat([df_new_machine_measurements,
                                         #right bundle branch block
                                          f_1(func_poisk_name("right bundle branch block",["right"],[] ),"CRBBB")])

In [139]:
df_new_machine_measurements = pd.concat([df_new_machine_measurements,
                                         #right bundle branch block
                                          f_1(func_poisk_name("left bundle branch block",[],["incomplete"]),"CLBBB")])

In [140]:
df_new_machine_measurements = pd.concat([df_new_machine_measurements,
                                         #right bundle branch block
                                          f_1(func_poisk_name("st",["non","changes"],["-t"]),"NST_")])

In [141]:
df_new_machine_measurements = pd.concat([df_new_machine_measurements,
                                         #right bundle branch block
                                          f_1(func_poisk_name(" t ",["abnorm"],[" st-","st","specific"]),"NDT")])

In [142]:
func_poisk_name("subendocardial",[],[] )

[]

In [145]:
df_new_machine_measurements.report.value_counts()

report
CRBBB    19480
LAFB     19364
LVH      14868
CLBBB    10606
1AVB      4472
NST_      3387
NDT       1845
RVH       1025
IRBBB      573
LPFB       410
ILBBB      406
ISCLA      310
2AVB       211
ILMI        33
WPW         17
ALMI        16
AMI          1
ISCIL        1
Name: count, dtype: int64

In [144]:
df_new_machine_measurements = df_new_machine_measurements.drop_duplicates()
scp_data_new["Unnamed_len"] = len(scp_data_new["Unnamed: 0"])*[0] 
for i in df_new_machine_measurements.report.value_counts().index:
    k_0 = scp_data_new[scp_data_new["Unnamed: 0"] == str(i)].index
    scp_data_new.Unnamed_len[k_0] =  int(df_new_machine_measurements.report.value_counts()[str(i)])

In [ ]:
#df_new_machine_measurements.to_csv("report_study_df_mimic_ver1.csv")
#df_new_machine_measurements = pd.read_csv('/kaggle/input/electrocardiogram-database-for-arrhythmia-study/report_study_df_mimic_ver1.csv', index_col=0)

In [146]:
df_new_machine_measurements.report_1.value_counts()[1:]

report_1
left bundle branch block                                                                         10198
left anterior fascicular block                                                                    9674
possible left anterior fascicular block                                                           8798
left ventricular hypertrophy                                                                      7466
possible left ventricular hypertrophy                                                             3559
                                                                                                 ...  
changes in v2 are probably due to left ventricular hypertrophy but consider septal infarction        1
inferior/lateral t wave changes suggest myocardial infarct                                           1
anterolateral st-t changes suggest myocardial infarct                                                1
inferior/lateral st-t changes suggest myocardial infarction     

In [147]:
df_new_machine_measurements.report.value_counts()[1:17]

report
LAFB     19364
LVH      14868
CLBBB    10606
1AVB      4472
NST_      3387
NDT       1845
RVH       1025
IRBBB      573
LPFB       410
ILBBB      406
ISCLA      310
2AVB       211
ILMI        33
WPW         17
ALMI        16
AMI          1
Name: count, dtype: int64

In [148]:
d_measurements = df_new_machine_measurements[df_new_machine_measurements.report == "CRBBB"][:2000]
for i in df_new_machine_measurements.report.value_counts()[2:17].index:
    d_measurements = pd.concat([
        d_measurements,
        df_new_machine_measurements[df_new_machine_measurements.report == i][:2000],
])


In [150]:
d_measurements.report.value_counts()

report
CRBBB    2000
LVH      2000
CLBBB    2000
1AVB     2000
NST_     2000
NDT      1845
RVH      1025
IRBBB     573
LPFB      410
ILBBB     406
ISCLA     310
2AVB      211
ILMI       33
WPW        17
ALMI       16
AMI         1
Name: count, dtype: int64

In [151]:
d_measurements

,report,report_1,study_id
10052926,CRBBB,incomplete right bundle branch block,43221168.0
10113525,CRBBB,incomplete right bundle branch block,44969762.0
10113525,CRBBB,incomplete right bundle branch block,44989745.0
10126957,CRBBB,incomplete right bundle branch block,40137514.0
10126957,CRBBB,incomplete right bundle branch block,45791599.0
...,...,...,...
10423888,ALMI,anterolateral t wave changes suggest myocardia...,49405512.0
13200333,ALMI,anterolateral t wave changes suggest myocardia...,48484702.0
16043920,ALMI,anterolateral t wave changes suggest myocardia...,45089637.0
10819535,ALMI,anterolateral st-t changes suggest myocardial ...,48122927.0


In [152]:
d_measurements["report_0"] = len(d_measurements)*["-"]

In [153]:
report_0_0=[]
for i in d_measurements.study_id:
    report_0_0.append(list(machine_measurements[machine_measurements.study_id == i].report_0)[0])

In [154]:
d_measurements["report_0"] = report_0_0

In [155]:
def freport_0(name):
    for j in ["sinus rhythm","sinus tachycardia","sinus bradycardia","atrial fibrillation","sinus arrhythmia","atrial flutter" ]:
        if j in str(name):
            return j
    return name

report_0_0 = []
for i in d_measurements.study_id:
    name = list(d_measurements[d_measurements.study_id == i]["report_0"])[0]
    report_0_0.append(freport_0(name))

In [156]:
d_measurements[:4]

,report,report_1,study_id,report_0
10052926,CRBBB,incomplete right bundle branch block,43221168.0,sinus rhythm
10113525,CRBBB,incomplete right bundle branch block,44969762.0,sinus rhythm
10113525,CRBBB,incomplete right bundle branch block,44989745.0,sinus rhythm
10126957,CRBBB,incomplete right bundle branch block,40137514.0,atrial fibrillation


## read record - создание файла с путями для предыдущей таблицы

In [157]:
record_list = pd.read_csv("/kaggle/input/electrocardiogram-database-for-arrhythmia-study/mimic/record_list.csv", index_col=0)

In [158]:
path = []
name_file = []
for i in d_measurements.study_id:
    n=  list(record_list[record_list.study_id == i].path)[0].split("/")
    path.append("/".join(n[:-1]))
    name_file.append("/".join(n[-1:]))

In [159]:
d_measurements["path"] = path
d_measurements["name_file"] = name_file

In [160]:
d_measurements[:4]

,report,report_1,study_id,report_0,path,name_file
10052926,CRBBB,incomplete right bundle branch block,43221168.0,sinus rhythm,files/p1005/p10052926/s43221168,43221168
10113525,CRBBB,incomplete right bundle branch block,44969762.0,sinus rhythm,files/p1011/p10113525/s44969762,44969762
10113525,CRBBB,incomplete right bundle branch block,44989745.0,sinus rhythm,files/p1011/p10113525/s44989745,44989745
10126957,CRBBB,incomplete right bundle branch block,40137514.0,atrial fibrillation,files/p1012/p10126957/s40137514,40137514


In [ ]:
#d_measurements.to_csv("d_measurements.csv")

In [162]:
record_list = pd.read_csv("/kaggle/input/electrocardiogram-database-for-arrhythmia-study/mimic/record_list.csv", index_col=0)
d_measurements = pd.read_csv("/kaggle/input/electrocardiogram-database-for-arrhythmia-study/d_measurements.csv", index_col=0)
ecg_df_HTV_mimic = pd.read_csv("/kaggle/input/electrocardiogram-database-for-arrhythmia-study/ecg_df_HTV_mimic.csv", index_col=0)
len(d_measurements),len(ecg_df_HTV_mimic)

(18811, 18811)